# RF Data Converter Introduction
----

<div class="alert alert-box alert-info">
Please use Jupyter labs http://board_ip_address/lab for this notebook.
</div>

This notebook presents an introduction to the RF Data
Converters (RF DCs) on the RFSoC2x2 board.

## Aims
* Describe the `xrfdc` Python package that is required to control and
  configure the RF DCs from Jupyter
* Investigate the radio hierarchy in the base overlay, allowing the
  user to develop very simple RF designs. 
* Present the data inspection and visualisation of the RF DCs using 
  Plotly.

## Table of Contents
* [Introduction](#introduction)
* [Hardware Setup](#hardware-setup)
* [The Radio Hierarchy](#radio-hierarchy)
* [Transmitter and Receiver](#the-transmitter-and-receiver)
* [RF Data Inspection](#rf-data-inspection)
* [Conclusion](#conclusion)

## References
* [Xilinx, Inc, "USP RF Data Converter: LogiCORE IP Product Guide", PG269, v2.3, June 2020](https://www.xilinx.com/support/documentation/ip_documentation/usp_rf_data_converter/v2_3/pg269-rf-data-converter.pdf)
* [Xilinx, Inc, "Vivado Design Suite: The AXI Reference Guide", UG1037, v4.0, June 2017](https://www.xilinx.com/support/documentation/ip_documentation/axi_ref_guide/latest/ug1037-vivado-axi-reference-guide.pdf)

## Revision History

* v1.0 | 02/02/2021 | First notebook revision.

----

## Introduction <a class="anchor" id="introduction"></a>

An overview of the Zynq RFSoC device is shown below; there are
three major components:

* Processing System (PS)
* Programmable Logic (PL)
* RF Data Converters (RF DC, including ADC and DAC)

The RF data converters are significant features of the Zynq RFSoC
device as they interface directly to the PL.
This brings many advantages, including the ability to
perform direct RF sampling and low-latency processing.

The RFSoC2x2 platform consists of a xczu28dr-ffvg1517 RFSoC device, 
which contains the following RF DCs:

* 8x RF Analogue-to-Digital Converters (RF ADCs)
    * Sample rate of 4 GSPS
    * 12-bit conversion
* 8x RF Digital-to-Analogue Converters (RF DACs)
    * Sample rate of 6.5 GSPS
    * 14-bit conversion

<br><img src='data/rfsystem_rfdc_overview.png' align='left' style='left' height='50%' width='50%'/>

Let's take a closer look at the RF DC block. Starting with
the RF ADC tiles, you will notice that there are 4 tiles on this
particular RFSoC device. Each tile contains 2 RF ADC blocks, 
which can be used to receive, or Digital Down Convert (DDC), 
an analogue signal. Each RF ADC block contains:

* A gearbox FIFO
* 2x, 4x, or 8x decimator
* A complex mixer
* A Quadrature Modulation Correction (QMC) unit

The image below presents the typical RF ADC processing 
pipeline:

1. The RF ADC samples the input waveform to convert it
   into a digital signal.
2. A threshold detector can be employed to detect and record
   input amplitude levels. 
3. The QMC is used to offset potential imbalance in the
   quality of the received complex signal.
4. The I and Q mixer can mix the input signal to baseband.
5. The I and Q decimators is capable of 2x, 4x, or 8x
   decimation before interfacing the PL via the gearbox FIFO.

<img src='data/rfsystem_rfadc_block_overview.png' align='left' style='left' height='75%' width='75%'/>

The RF DAC block has a similar pipeline to the RF ADC;
however, it is in reverse this time for Digital Up Conversion (DUC).

1. The data to be transmitted is interfaced to the RF DAC block from the PL.
2. The digital signal is interpolated by a factor of 2x, 4x, or 8x.
3. The digital signal is transferred to the complex mixer to be mixed
   to the desired carrier frequency. 
4. After the QMC and coarse delay block, the signal can optionally
   be convolved with an inverse (anti) sinc filter to improve the
   roll-off of the first Nyquist zone.
5. The digital signal is then converted to analogue through the
   RF DAC sampler.


<img src='data/rfsystem_rfdac_block_overview.png' align='left' style='left' height='75%' width='75%'/>

In this notebook, we will be demonstrating many of the features
mentioned above via a simple loopback example.

## Hardware Setup <a class="anchor" id="hardware-setup"></a>

Your RFSoC2x2 board is dual-channel. Notice that there are only
4 SMAs on your board, labeled DAC1, DAC2, ADC1, and ADC2.
Only two sets of RF ADC and RF DAC channels are accessible on
the RFSoC2x2 board.

You should create the loopback connection using SMA cables
as shown below:

* Channel 0: DAC2 to ADC2
* Channel 1: DAC1 to ADC1

See the image below for a demonstration.

<img src='data/rfsoc2x2_connections.jpg' align='left' style='left' width='40%' height='40%'/>

<div class="alert alert-heading alert-danger">
    <h4 class="alert-heading">Warning:</h4>

In this demo we are transmitting signals via cables.
This device can also transmit wireless signals. 
Keep in mind that unlicensed transmission of wireless signals
may be illegal in your location. 
Radio signals may also interfere with nearby devices,
such as pacemakers and emergency radio equipment. 
If you are unsure, please seek professional support.
</div>

----

## The Radio Hierarchy <a class="anchor" id="radio-hierarchy"></a>

We need to add custom control logic in the PL to communicate
with the RF ADC or RF DAC. The RFSoC2x2 base overlay contains
such logic that will allow you to evaluate the RF DCs. 
There are two IP Cores - the transmitter and the receiver.

* **Amplitude Controller** - can apply a value to the input of the RF DAC.
The value can be set by writing to an AXI-Lite register. 
The register value can be mixed with the Numerically
Controlled Oscillator (NCO) in the fine mixer to create a tone. 
The tone is looped back round into the RF ADC.

* **Packet Generator** - interfaces to the output of RF ADC block. 
The data is used to generate an [AXI-Stream packet](https://www.xilinx.com/support/documentation/ip_documentation/axi_ref_guide/latest/ug1037-vivado-axi-reference-guide.pdf).
The AXI-Stream packet is sent to an AXI Direct Memory
Access (DMA) core and transferred to DDR memory.

These 2 IP cores are connected to each channel of the RFSoC2x2 board.
They are conveniently placed inside a hierarchical block, `radio`, 
as shown below.

<img src='data/rfsystem_introduction.png' align='left' style='left' height='60%' width='60%'/>

Let's now download the base overlay and initialize the drivers.

In [1]:
from pynq.overlays.base import BaseOverlay

base = BaseOverlay('base.bit')

The RFSoC2x2 has a sophisticated clocking network, which can generate
low-jitter clocks for the RF DC Phase-Locked Loops (PLLs). The base overlay
has a simple method to initialize these clocks. Run the cell below to set
the LMK and LMX clocks to 122.88MHz and 409.6MHz, respectively.

In [2]:
base.init_rf_clks()

Now we should investigate the `radio` hierarchy.

In [3]:
base.radio?

Type:           RadioWrapper
String form:    <rfsystem.hierarchies.RadioWrapper object at 0x7f68e29e10>
File:           /usr/local/lib/python3.6/dist-packages/rfsystem/hierarchies.py
Docstring:     
Hierarchy driver for the entire radio subsystem.

Exposes the transmit, receive, and rfdc as attributes. The rfdc
dac and adc tiles are passed to the transmitter and receiver
respectively.

This wrapper assumes the following separate pipelines
* transmitter -> rfdc -> "the outside world"
* "the outside world" -> rfdc => receiver

The rfdc core is common between each pipeline.

Attributes
----------
transmitter : pynq.lib.radio.Transmitter
    The transmit control system
receiver : pynq.lib.radio.Receiver
    The receive control system
rfdc : xrfdc package
    The RF Data Converter core
Init docstring:
Create a new _IPMap based on a hierarchical description.

        


We can see the `transmitter` and `receiver` hierarchies. 
The RF DC object is also available, however, we don't need this. The reason
is that each transmitter and receiver is automatically paired with their
associated RF DC tile and block.

The `radio` hierarchy also initializes all of the active RF ADC and RF DAC
blocks to sample at 4096 GSPS (the maximum sample rate for the RF ADC).
By default the RF DAC and RF ADC mixer frequencies are 1024 MHz, which is
in the center of the first Nyquist Zone.

## Transmitter and Receiver <a class="anchor" id="the-transmitter-and-receiver"></a>

The transmitter hierarchy contains two channels. Each channel contains an
*Amplitude Controller* connected directly to an RF DAC. We can list the
available channels of the transmitter using the `get_channel_description()`
method.

In [4]:
base.radio.transmitter.get_channel_description()

{'Channel 0': {'Block': 0, 'Tile': 228},
 'Channel 1': {'Block': 0, 'Tile': 229}}

The report above states that there are two active channels in the system.

The receiver hierarchy also contains two channels. The *Packet Generator*
arranges the data into AXI-Stream packets and sends these packets to DDR
memory via AXI DMA. Let's run the cell below.

In [5]:
base.radio.receiver.get_channel_description()

{'Channel 0': {'Block': 0, 'Tile': 224},
 'Channel 1': {'Block': 0, 'Tile': 226}}

There are several methods to read and write to RF block registers.
These methods are common to the RF DAC and RF ADC.

```python
BlockStatus
MixerSettings
QMCSettings
CoarseDelaySettings
NyquistZone
```

For example, `base.radio.transmitter.channel[0].dac_block.QMCSettings`
will display the QMC settings for the transmit DAC block.

For many of these settings, you can change them and they will impact the operation.

In [6]:
base.radio.transmitter.channel[0].dac_block.QMCSettings

{'EnableGain': 0,
 'EnablePhase': 0,
 'EventSource': 0,
 'GainCorrectionFactor': 0.0,
 'OffsetCorrectionFactor': 0,
 'PhaseCorrectionFactor': 0.0}

Another example you can try:
```python
base.radio.receiver.channel[0].adc_block.CoarseDelaySettings
```

To further investigate the RF ADC and RF DAC, you can run the code cell below.

In [7]:
help(base.radio.transmitter.channel[0].dac_block)

Help on RFdcDacBlock in module xrfdc object:

class RFdcDacBlock(RFdcBlock)
 |  Method resolution order:
 |      RFdcDacBlock
 |      RFdcBlock
 |      builtins.object
 |  
 |  Methods defined here:
 |  
 |  __init__(self, *args, **kwargs)
 |      Initialize self.  See help(type(self)) for accurate signature.
 |  
 |  ----------------------------------------------------------------------
 |  Data descriptors defined here:
 |  
 |  DecoderMode
 |  
 |  FabRdVldWords
 |  
 |  FabWrVldWords
 |  
 |  InterpolationFactor
 |  
 |  InvSincFIR
 |  
 |  OutputCurr
 |  
 |  ----------------------------------------------------------------------
 |  Methods inherited from RFdcBlock:
 |  
 |  ResetNCOPhase(self)
 |  
 |  UpdateEvent(self, Event)
 |  
 |  ----------------------------------------------------------------------
 |  Data descriptors inherited from RFdcBlock:
 |  
 |  BlockStatus
 |  
 |  CoarseDelaySettings
 |  
 |  MixerSettings
 |  
 |  NyquistZone
 |  
 |  QMCSettings
 |  
 |  __dict

More to try:
```python
help(base.radio.transmitter.channel[0].dac_tile)
help(base.radio.receiver.channel[0].adc_block)
help(base.radio.receiver.channel[0].adc_tile)
```
Many of these attributes / methods depend on the configuration
of the PL and the hardware. You will need to implement your own hardware 
design if you want to do something else.

## RF Data Inspection <a class="anchor" id="rf-data-inspection"></a>

Now that you have a fundamental understanding of how to control the RF DCs, 
we can inspect RF data using the radio hierarchy block. To make it more
interesting, we generate stimulus using the RF DAC block for each channel. 
Let's configure the mixer frequencies for each RF-DAC block channel below.

In [8]:
base.radio.transmitter.channel[0].dac_block.MixerSettings['Freq'] = 1000 # MHz
base.radio.transmitter.channel[1].dac_block.MixerSettings['Freq'] = 900 # MHz

The PL contains an amplitude controller for signal transmission. 
We can set the amplitude of the signal and enable transmission for each channel.

In [9]:
for i in range(0, len(base.radio.transmitter.channel)):
    base.radio.transmitter.channel[i].control.gain = 0.5
    base.radio.transmitter.channel[i].control.enable = True

The RF-ADC block mixer frequency can also be tuned. We set it to 1024 MHz as
that is the default setting. This is a suitable mixer frequency for the
next part of the demonstration, where we will transfer time domain samples
from receiver channels.

Firstly, we will define the numbers of samples to be transferred. 
The range of samples that can be transferred is between 16 to 32768.
Running the code cell below, will transfer complex data, with an amplitude of 1.

In [10]:
number_samples = 256
cdata = []
for i in range(0, len(base.radio.receiver.channel)):
    cdata.append(base.radio.receiver.channel[i].transfer(number_samples))

Now that we have complex channel data in Jupyter, we can import plotly, numpy,
and ipwidgets to enable data visualization.

In [11]:
import plotly.graph_objs as go
import numpy as np
import ipywidgets as ipw

The data can be plotted using `go.Scatter` and `go.FigureWidget` for each channel. Ipywidgets allows each figure to be displayed at the same time.

In [12]:
sample_frequency = 2048e6
figs = []

for i in range(0, len(base.radio.transmitter.channel)):
    plt_re_temp = (go.Scatter(
            x = np.arange(0, number_samples/sample_frequency, 
                          1/sample_frequency),
            y = np.real(cdata[i]), name='Real'))
    plt_im_temp = (go.Scatter(
            x = np.arange(0, number_samples/sample_frequency, 
                          1/sample_frequency),
            y = np.imag(cdata[i]), name='Imag'))
    figs.append(go.FigureWidget(data = [plt_re_temp, plt_im_temp],
            layout = {'title': ''.join(['Time Domain Plot of ADC Channel ', 
                                         str(i)]), 
                      'xaxis': {'title' : 'Seconds (s)',
                                'autorange' : True},
                      'yaxis' : {'title' : 'Amplitude (V)'}}))

ipw.VBox(figs)

    'data': [{'name': 'Real',
              'type': 'scatter',
              'ui…

You can also return to the [RF Data Inspection](#rf-data-inspection) section
and try different mixer frequency or different amplitude gain.

## Conclusion <a class="anchor" id="conclusion"></a>

This notebook has presented a simple introduction to the RF Data Converters. 
The RF DCs were briefly described and a radio hierarchy in PL was explored.
RF data visualization was performed using `plotly`, `ipywidgets`, and `numpy` in Jupyter.

| | [RF Spectrum Analysis ➡️](02_rf_spectrum_analysis.ipynb)

Copyright (C) 2021 Xilinx, Inc

SPDX-License-Identifier: BSD-3-Clause

----

----